First, let's load the JSON file which describes the human pose task.  This is in COCO format, it is the category descriptor pulled from the annotations file.  We modify the COCO category slightly, to add a neck keypoint.  We will use this task description JSON to create a topology tensor, which is an intermediate data structure that describes the part linkages, as well as which channels in the part affinity field each linkage corresponds to.

In [1]:
import sys

sys.path.insert(0, "~/trt_pose/")

In [2]:
sys.path

['~/trt_pose/',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/home/jetbot/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/torchvision-0.4.0a0+d31eafa-py3.6-linux-aarch64.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_SSD1306-1.6.2-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_MotorHAT-1.4.0-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_GPIO-1.0.4-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/spidev-3.4-py3.6-linux-aarch64.egg',
 '/usr/local/lib/python3.6/dist-packages/Adafruit_PureIO-1.0.4-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/trt_pose-0.0.1-py3.6-linux-aarch64.egg',
 '/usr/local/lib/python3.6/dist-packages/torch2trt-0.2.0-py3.6.egg',
 '/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/l

In [1]:
import json
import trt_pose.coco
import trt_pose

print(trt_pose.__path__)
with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)
# tag: [7, 9] [8, 10]
# tag in topology [6, 8] [7, 9]
# tag index 6. 7

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-vsv4fmfq because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


['/home/jetbot/trt_pose/tasks/human_pose/trt_pose']
[[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7], [18, 1], [18, 6], [18, 7], [18, 12], [18, 13]]
tensor([[ 0,  1, 15, 13],
        [ 2,  3, 13, 11],
        [ 4,  5, 16, 14],
        [ 6,  7, 14, 12],
        [ 8,  9, 11, 12],
        [10, 11,  5,  7],
        [12, 13,  6,  8],
        [14, 15,  7,  9],
        [16, 17,  8, 10],
        [18, 19,  1,  2],
        [20, 21,  0,  1],
        [22, 23,  0,  2],
        [24, 25,  1,  3],
        [26, 27,  2,  4],
        [28, 29,  3,  5],
        [30, 31,  4,  6],
        [32, 33, 17,  0],
        [34, 35, 17,  5],
        [36, 37, 17,  6],
        [38, 39, 17, 11],
        [40, 41, 17, 12]], dtype=torch.int32)


In [2]:
print(topology.shape[0])

21


Next, we'll load our model.  Each model takes at least two parameters, *cmap_channels* and *paf_channels* corresponding to the number of heatmap channels
and part affinity field channels.  The number of part affinity field channels is 2x the number of links, because each link has a channel corresponding to the
x and y direction of the vector field for each link.

In [3]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()

Next, let's load the model weights.  You will need to download these according to the table in the README.

In [4]:
import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))

<All keys matched successfully>

In order to optimize with TensorRT using the python library *torch2trt* we'll also need to create some example data.  The dimensions
of this data should match the dimensions that the network was trained with.  Since we're using the resnet18 variant that was trained on
an input resolution of 224x224, we set the width and height to these dimensions.

In [5]:
WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

Next, we'll use [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) to optimize the model.  We'll enable fp16_mode to allow optimizations to use reduced half precision.

In [6]:
import torch2trt

model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)

The optimized model may be saved so that we do not need to perform optimization again, we can just load the model.  Please note that TensorRT has device specific optimizations, so you can only use an optimized model on similar platforms.

In [7]:
OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

We could then load the saved model using *torch2trt* as follows.

In [8]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

We can benchmark the model in FPS with the following code

In [9]:
import time

t0 = time.time()
torch.cuda.current_stream().synchronize()
for i in range(50):
    y = model_trt(data)
torch.cuda.current_stream().synchronize()
t1 = time.time()

print(50.0 / (t1 - t0))

12.660554623707533


Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format.

In [10]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Next, we'll define two callable classes that will be used to parse the objects from the neural network, as well as draw the parsed objects on an image.

In [11]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

Assuming you're using NVIDIA Jetson, you can use the [jetcam](https://github.com/NVIDIA-AI-IOT/jetcam) package to create an easy to use camera that will produce images in BGR8/HWC format.

If you're not on Jetson, you may need to adapt the code below.

In [12]:
# from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

# camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

Next, we'll create a widget which will be used to display the camera feed with visualizations.

In [13]:
import ipywidgets
from IPython.display import display

image_w = ipywidgets.Image(format='jpeg')

display(image_w)

Image(value=b'', format='jpeg')

Finally, we'll define the main execution loop.  This will perform the following steps

1.  Preprocess the camera image
2.  Execute the neural network
3.  Parse the objects from the neural network output
4.  Draw the objects onto the camera image
5.  Convert the image to JPEG format and stream to the display widget

In [14]:
# counts, objects, peaks = None
def execute(change):
#     global counts, objects, peaks
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    x0, y0, x1, y1 = draw_objects(image, counts, objects, peaks)
#     print(objects)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    print("(%.2f, %.2f), (%.2f, %.2f)" % (x0, y0, x1, y1))

In [ ]:
# todo:
# draw_objects.capture()

If we call the cell below it will execute the function once on the current camera frame.

In [15]:
execute({'new': camera.value})

(171.00, 26.00), (167.00, 58.00)


Call the cell below to attach the execution function to the camera's internal value.  This will cause the execute function to be called whenever a new camera frame is received.

In [16]:
camera.observe(execute, names='value')

(140.00, 85.00), (135.00, 97.00)
(139.00, 85.00), (134.00, 94.00)
(139.00, 85.00), (134.00, 94.00)
(154.00, 90.00), (153.00, 113.00)
(139.00, 85.00), (135.00, 96.00)
(154.00, 90.00), (153.00, 113.00)
(155.00, 90.00), (151.00, 107.00)
(140.00, 84.00), (137.00, 98.00)
(154.00, 90.00), (154.00, 113.00)
(139.00, 85.00), (135.00, 98.00)
(138.00, 85.00), (135.00, 98.00)
(139.00, 85.00), (136.00, 98.00)
(139.00, 86.00), (136.00, 98.00)
(139.00, 86.00), (136.00, 97.00)
(154.00, 90.00), (153.00, 113.00)
(139.00, 85.00), (136.00, 98.00)
(139.00, 85.00), (136.00, 99.00)
(139.00, 86.00), (135.00, 98.00)
(154.00, 91.00), (153.00, 113.00)
(154.00, 91.00), (152.00, 113.00)
(139.00, 87.00), (135.00, 99.00)
(153.00, 90.00), (153.00, 113.00)
(154.00, 90.00), (153.00, 113.00)
(138.00, 86.00), (135.00, 99.00)
(154.00, 90.00), (154.00, 114.00)
(138.00, 85.00), (136.00, 99.00)
(138.00, 85.00), (135.00, 99.00)
(138.00, 86.00), (135.00, 100.00)
(138.00, 85.00), (135.00, 101.00)
(138.00, 85.00), (135.00, 99.00

Call the cell below to unattach the camera frame callbacks.

In [17]:
camera.unobserve_all()

(148.00, 79.00), (145.00, 103.00)
(148.00, 79.00), (144.00, 105.00)
